# Laster inn nødvendige pakker

library(readr)
library(ggplot2)
library(tidyverse)
library(data.table)
library(zoo)
library(lubridate)
library(cowplot)
#importing table

Oppgave 1

# Bruker readlines til å lese inn datasettet og etterhvert lage det til et datasett er numeric - noe som vi kan lage et plott ut av.

ds <- readLines("https://www.nsstc.uah.edu/data/msu/v6.0/tlt/uahncdc_lt_6.0.txt")
ds

# Fjerner tolv linjer slutten av tabellen - unødvendige linjer med skrift.

ds1 <- head(ds, -12)

# Bruker read_table til å lage datasettet om til en tabell.

df1 <- read_table(ds1)

# Velger ut tre variablene Year, Mo og Globe siden det er de vi skal lage et plott av. Har en tabell nå som består av 517 observasjoner og 3 variabler.

df2 <- df1 %>% 
  select(Year, Mo, Globe)
exists("df1")

temp_globe_year <- df2 %>%
  select(Year, Globe, Mo) %>% 
  mutate(Date = paste(Year, Mo, sep = "-")) %>%
  mutate(Date = lubridate::ym(Date)) %>%
  mutate(average_temp = zoo::rollmean(Globe, 13, 
                                      fill = NA, align = "center"))

# Tekst til en tekstboks midt i første plott.

text_box_label = "UAH Satelite based\nTemperature of the\nGlobal Lower Atmosphere\n(Version 6.0)"

# Lager selve plottet som skal være tilnærmet lik det som var på nettsiden.

plot <-temp_globe_year %>% 
  ggplot(aes(x = Date)) +
  geom_hline(yintercept = 0) +   # add line at 0 
  # add points and line:
  geom_point(aes(y = Globe), colour = "blue4", shape = 21) + # colour should be outside the aes as it is not based on a column
  geom_line(aes(y = Globe), colour = "blue4", alpha = 0.5) +
  # add average:
  geom_line(aes(y = average_temp, group = 1), 
            colour = "red", size = 1)  +
  scale_y_continuous(breaks = seq(from= -0.7,to=0.9, by = 0.1) , 
                     labels = scales::comma) +  
  scale_x_date(date_breaks = "year", date_labels = "%Y",
               expand = c(0,0.1)) + 
  labs(title = "Latest Global Average Tropospheric Temperatures",
       x = NULL,
       y = "Departure from '91-'20 Avg. (deg. C)") +
  theme_bw() +
  annotate(geom="text", x=as.Date("2004-01-01"), y=-0.5, 
           label="Running, centered\n13 month average", 
           colour = "red") + 
  geom_segment(x = as.Date("2004-01-01"), y=-0.45,
               xend = as.Date("2008-01-01"), yend=-0.2,
               arrow = arrow(angle = 20, type = "closed",
                             length = unit(0.15, "inches")),
               colour = "red", size = 1) +
  annotate(geom="text", 
           x=as.Date("1987-01-01"), 
           y = 0.5, hjust = 0.5,
           label = text_box_label,
           colour = "blue4" ) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5),
        panel.grid.minor.y = element_blank())


plot




#Oppgave 2

# Laster inn datasett for alle de fire ulike for å lage fire plott som viser nedre troposfære, midtre troposfære, troposfæren og nedre stratosfære

Lower_Trop <- fread("https://www.nsstc.uah.edu/data/msu/v6.0/tlt/uahncdc_lt_6.0.txt", nrows = 517)
Lower_Trop1 <- Lower_Trop %>%
  select(Year, NoPol, Mo,) %>% 
  mutate(Date = paste(Year, Mo, sep = "-")) %>%
  mutate(Date = lubridate::ym(Date)) %>%
  mutate(average_temp = zoo::rollmean(NoPol, 13, 
                                      fill = NA, align = "center"))



Mid_Trop <- fread("https://www.nsstc.uah.edu/data/msu/v6.0/tmt/uahncdc_mt_6.0.txt", nrows = 517)
Mid_Trop1 <- Mid_Trop %>%
  select(Year, NoPol, Mo,) %>%  
  mutate(Date = paste(Year, Mo, sep = "-")) %>%
  mutate(Date = lubridate::ym(Date)) %>%
  mutate(average_temp = zoo::rollmean(NoPol, 13, 
                                      fill = NA, align = "center"))



Trop <- fread("https://www.nsstc.uah.edu/data/msu/v6.0/ttp/uahncdc_tp_6.0.txt", nrows = 517)
Trop1 <- Trop %>%
  select(Year, NoPol, Mo,) %>% 
  mutate(Date = paste(Year, Mo, sep = "-")) %>%
  mutate(Date = lubridate::ym(Date)) %>%
  mutate(average_temp = zoo::rollmean(NoPol, 13, 
                                      fill = NA, align = "center"))



Lower_Strat <- fread("https://www.nsstc.uah.edu/data/msu/v6.0/tls/uahncdc_ls_6.0.txt" , nrows = 517)
Lower_Strat1 <- Lower_Strat %>%
  select(Year, NoPol, Mo,) %>%  
  mutate(Date = paste(Year, Mo, sep = "-")) %>%
  mutate(Date = lubridate::ym(Date)) %>%
  mutate(average_temp = zoo::rollmean(NoPol, 13, 
                                      fill = NA, align = "center"))


# Plott til hver og en av datasettene:
# Plottene viser gjennomsnittstemperaturen, hvor vi har brukt zoo::rollmean - som regner ut et 13 måneders glidende gjennomsnitt.

p1 <- Lower_Trop1 %>%
  ggplot(aes(x = Date, y = NoPol)) +
  geom_line(aes(y = average_temp, group = 1), 
            colour = "red", size = 1)  +
  geom_line(col = "Blue") +
  geom_point(col = "Blue") +
  labs(title = "Nedre troposfære",
       x = " ",
       y = "Temperatur")
p2 <- Mid_Trop1 %>%
  ggplot(aes(x = Date, y = NoPol)) +
  geom_line(aes(y = average_temp, group = 1), 
            colour = "red", size = 1)  +
  geom_line(col = "blue") +
  geom_point(col = "blue") +
  labs(title = "Midt troposfære",
       x = " ",
       y = "Temperatur")
p3 <- Trop1 %>%
  ggplot(aes(x = Date, y = NoPol)) +
  geom_line(aes(y = average_temp, group = 1), 
            colour = "red", size = 1)  +
  geom_line(col = "Blue") +
  geom_point(col = "blue") +
  labs(title = "Troposfære",
       x = " ",
       y = "Temperatur")
p4 <- Lower_Strat1 %>%
  ggplot(aes(x = Date, y = NoPol)) +
  geom_line(aes(y = average_temp, group = 1), 
            colour = "red", size = 1)  +
  geom_line(col = "purple") +
  geom_point(col = "purple") +
  labs(title = "Nedre stratosfære",
       x = " ",
       y = "Temperatur")
plot_grid(p1, p2, p3, p4, ncol = 2, labels = "AUTO")